# 텍스트를 읽고 긍정, 부정 예측하기

In [ ]:
# -*- coding: utf-8 -*-
# 코드 내부에 한글을 사용가능 하게 해주는 부분입니다.

import numpy
import tensorflow as tf
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding


#주어진 문장을 '단어'로 토큰화 하기

#케라스의 텍스트 전처리와 관련한 함수중 text_to_word_sequence 함수를 불러 옵니다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 텍스트의 토큰화

In [ ]:
# 전처리할 텍스트를 정합니다.
text = '해보지 않으면 해낼 수 없다'
 
# 해당 텍스트를 토큰화 합니다.
result = text_to_word_sequence(text)
print("\n원문:\n", text)
print("\n토큰화:\n", result)
 
#단어 빈도수 세기

#전처리 하려는 세개의 문장을 정합니다.
# 전처리하려는 세 개의 문장을 docs라는 배열에 지정함

docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
       '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다.',
       '토큰화 한 결과는 딥러닝에서 사용 할 수 있습니다.',
       ]
# 케라스의 Tokenizer() 함수를 사용하면 단어의 빈도 수를 쉽게 계산할 수 있음
# 다음 예제는 위에 제시된 세 문장의 단어를 빈도 수로 다시 정리하는 코드 
# 토큰화 함수를 이용해 전처리 하는 과정입니다.
token = Tokenizer()             # 토큰화 함수 지정
token.fit_on_texts(docs)       # 토큰화 함수에 문장 적용
 
#단어의 빈도수를 계산한 결과를 각 옵션에 맞추어 출력합니다. 
#word_counts는 단어의 빈도 수를 계산해주는 함수 
print("\n단어 카운트:\n", token.word_counts) 

#Tokenizer()의 word_counts 함수는 순서를 기억하는 OrderedDict클래스를 사용합니다.
#순서를 기억하는 OrderedDict 클래스에 담겨 있는 형태로 출력되는 것을 볼 수 있음
#document_count( ) 함수를 이용하면 총 몇 개의 문장이 들어있는지를 셀 수도 있음
#출력되는 순서는 랜덤입니다. 
print("\n문장 카운트: ", token.document_count)
#word_docs() 함수를 통해 각 단어들이 몇 개의 문장에 나오는가를 세어서 출력할 수도 있음
#출력되는 순서는 랜덤임
print("\n각 단어가 몇개의 문장에 포함되어 있는가:\n", token.word_docs)
#각 단어에 매겨진 인덱스 값을 출력하려면 word_index( ) 함수를 사용하면 됨
print("\n각 단어에 매겨진 인덱스 값:\n",  token.word_index)


원문:
 해보지 않으면 해낼 수 없다

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']

단어 카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 3), ('합니다', 1), ('단어로', 1), ('해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('한', 1), ('결과는', 1), ('사용', 1), ('할', 1), ('수', 1), ('있습니다', 1)])

문장 카운트:  3

각 단어가 몇개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'각': 1, '토큰화': 3, '단어를': 1, '텍스트의': 2, '나누어': 1, '합니다': 1, '먼저': 1, '인식됩니다': 1, '딥러닝에서': 2, '단어로': 1, '해야': 1, '한': 1, '있습니다': 1, '결과는': 1, '수': 1, '사용': 1, '할': 1})

각 단어에 매겨진 인덱스 값:
 {'토큰화': 1, '텍스트의': 2, '딥러닝에서': 3, '먼저': 4, '각': 5, '단어를': 6, '나누어': 7, '합니다': 8, '단어로': 9, '해야': 10, '인식됩니다': 11, '한': 12, '결과는': 13, '사용': 14, '할': 15, '수': 16, '있습니다': 17}


# 단어임베딩
먼저 짧은 리뷰 10개를 불러와 각각 긍정이면 1이라는 클래스를, 부정적이면 0이라는 클래스로 지정함 <br>
케라스에서 제공하는 Tokenizer( ) 함수의 fit_on_text를 이용해 각 단어를 하나의 토큰으로 변환함

In [ ]:
# 텍스트 리뷰 자료를 지정합니다.
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정합니다.
classes = array([1,1,1,1,1,0,0,0,0,0])

# 토큰화 
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


단어가 문장의 다른 요소와 어떤 관계를 가지고 있는지를 알아보는 방법이  필요함 <br>
이러한 기법 중에서 가장 기본적인 방법인 원-핫 인코딩(one-hotencoding)을 알아보자 <br>
각 단어를 모두 0으로 바꾸어 주고 원하는 단어만 1로 바꾸어 주는 것이 원-핫 인코딩이었음 <br>
이를 수행하기 위해 먼저 단어 수만큼 0으로 채워진 벡터 공간으로 바꾸면 다음과 같음 <br>
![image.png](attachment:image.png)

이제 토큰에 지정된 인덱스로 새로운 배열을 생성함.
그럼 주어진 텍스트는 숫자로 이루어진 다음과 같은 배열로 재편됨.


입력된 리뷰 데이터의 토큰 수가 각각 다르다는 것에 유의하시기 바람.
딥러닝 모델에 입력을 하려면 학습 데이터의 길이가 동일해야 함.

파이썬 배열의 인덱스는 0부터 시작하므로, 맨 앞에 0이 추가되는 것에 주의함 <br>
이제 각 단어가 배열 내에서 해당하는 위치를 1로 바꿔서 벡터화할 수 있음 <br>
![image.png](attachment:image.png)

In [ ]:
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


![image.png](attachment:image.png)

In [ ]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞추어 줍니다.
padded_x = pad_sequences(x, 4)  
print("\n패딩 결과:\n", padded_x)
 
#딥러닝 모델
print("\n딥러닝 모델 시작:")

#임베딩에 입력될 단어의 수를 지정합니다.
word_size = len(token.word_index) +1

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}

리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]

패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]

딥러닝 모델 시작:


원-핫 인코딩을 그대로 사용하면 벡터의 길이가 너무 길어진다는 단점이 있음 <br>

![image.png](attachment:image.png)

공간적 낭비를 해결하기 위해 등장한 것이 단어 임베딩(word embedding)이라는 방법 <br>
단어 임베딩은 주어진 배열을 정해진 길이로 압축시킴 <br>


이 과정은 케라스에서 제공하는 Embedding( ) 함수를 사용하면 간단히 해낼 수 있음 <br>
임베딩 함수에 필요한 세 가지 파라미터는 ‘입력, 출력, 단어 수’임 <br>
총 몇 개의 단어 집합에서(입력), 몇 개의 임베딩 결과를 사용할 것인지(출력), 그리고 매번 입력될 단어 수는 몇 개로 할지(단어 수)를 정해야 하는 것
![image.png](attachment:image.png)

![image.png](attachment:image.png)

몇 개의 임베딩 결과를 사용할 것인지, 즉 ‘출력’을 정할 차례임
word_size만큼의 입력 값을 이용해 8개의 임베딩 결과를 만듦
여기서 8이라는 숫자는 임의로 정한 것
데이터에 따라 적절한 값으로 바꿀 수 있음
이때 만들어진 8개의 임베딩 결과는 우리 눈에 보이지 않음
내부에서 계산하여 딥러닝의 레이어로 활용됨
끝으로 매번 입력될 ‘단어 수’를 정함
패딩 과정을 거쳐 4개의 길이로 맞춰 주었으므로 4개의 단어가 들어가게 설정하면 임베딩 과정은 다음 한 줄로 표현됨
![image.png](attachment:image.png)

In [ ]:
#단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력합니다.
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))

In [ ]:
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 0s 0s/step - loss: 0.6925 - accuracy: 0.6000
Epoch 2/20
1/1 [==============================] - 0s 0s/step - loss: 0.6905 - accuracy: 0.6000
Epoch 3/20
1/1 [==============================] - 0s 0s/step - loss: 0.6886 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 0s/step - loss: 0.6867 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 0s/step - loss: 0.6848 - accuracy: 0.7000
Epoch 6/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6829 - accuracy: 0.8000
Epoch 7/20
1/1 [==============================] - 0s 0s/step - loss: 0.6809 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 0s/step - loss: 0.6790 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6771 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6752 - accuracy: 0.8000
Epoch 11/20
1/1 [=============================